# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cantidad de oraciones: 54607542. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [39]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import gensim
import os
import random
import itertools
from multiprocessing import cpu_count, Pool
from gensim.models import Word2Vec, KeyedVectors

from corpus import spread_artID
from embeddings import concat_vectors, mean_vectors, fractional_decay, exponential_decay

### Cargamos los datos

In [ ]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

In [ ]:
print(word_mapping.shape)
word_mapping.head()

In [ ]:
len(word_mapping['word'].unique())

### Cargamos los artículos del Documento "0"

In [ ]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [ ]:
doc_0.shape

In [ ]:
doc_0.head()

Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [ ]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [ ]:
doc_0 = doc_0.dropna()

In [ ]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [ ]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [ ]:
doc_0.head()

Nos quedamos con un artículo

In [ ]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

Reconstruimos la primera oración del artículo

In [ ]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [ ]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [ ]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [12]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 35.06152105331421


In [13]:
print('Cantidad de word embeddings: {}'.format(len(w2v_model.vectors)))

Cantidad de word embeddings: 3000000


In [14]:
print('Dimensionalidad de los vectores: {}'.format(w2v_model.vector_size))

Dimensionalidad de los vectores: 300


## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google se tiene $D$ = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$
   

<br>
Al tomar una ventana simétrica, se realiza un relleno (padding) con ceros a izquierda y/o derecha según corresponda para mantener la misma dimesionalidad en cada nuevo vector generado.

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [ ]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [ ]:
print(coarseNE_0.shape)
coarseNE_0.head()

Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [ ]:
coarseNE_0 = spread_artID(coarseNE_0)
coarseNE_0 = coarseNE_0.dropna()

In [ ]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [ ]:
def get_tag(x):
    tags = ['PER', 'LOC', 'ORG', 'MISC']
    return tags[x]

In [ ]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [ ]:
coarseNE_0.head()

No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

Notar que esto sucede porque esos artículos no contienen entidades nombradas.

In [ ]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

In [ ]:
article = doc_0[doc_0.art_ID == 431177]
for sentence in article.sentence.values:
    dec_sentence = sentence_decoder(sentence, word_mapping)
    print(' '.join(dec_sentence))

### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [ ]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

In [ ]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]

In [ ]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

## Ahora utilicemos los dataframes preprocesados

In [15]:
doc77_df = pd.read_pickle('./corpus_WiNER/docs_df/doc_77')
article_df = doc77_df[doc77_df.art_ID == 145492]
dec_sentence = article_df.iloc[0, article_df.columns.get_loc('sentence')]
doc77_df.head()

,sentence,art_ID
1,"[Belton, House, is, a, Grade, I, listed, count...",145492.0
2,"[The, mansion, is, surrounded, by, formal, gar...",145492.0
3,"[Belton, has, been, described, as, a, compilat...",145492.0
4,"[The, house, has, also, been, described, as, t...",145492.0
5,"[Only, Brympton, d'Evercy, has, been, similarl...",145492.0


In [16]:
coarseNE_77 = pd.read_pickle('./corpus_WiNER/coarseNE_df/coarseNE_77')
art_entities_df = coarseNE_77[coarseNE_77.art_ID == 145492]
sen_entities_0 = art_entities_df[art_entities_df.senIdx == 0]
art_entities_df.head()

,art_ID,senIdx,begin,end,entityType
42515,145492.0,0,0,2,LOC
42516,145492.0,0,10,11,LOC
42517,145492.0,0,12,13,LOC
42518,145492.0,0,14,15,LOC
42519,145492.0,0,16,17,LOC


In [17]:
def entityListFromSentence(senIdx, sen_length, art_entities_df):
    # We take the df with the entities of each sentence
    sen_entities_df = art_entities_df[art_entities_df.senIdx == senIdx]
    # An empty dataframe means that the sentence doesn't have any entity
    if sen_entities_df.empty:
        entities = ['O' for _ in range(sen_length)]
    else:
        entities = []
        i = 0
        for _, row in sen_entities_df.iterrows():
            while i < row['begin']:
                entities.append('O')
                i += 1
            while i < row['end']:
                entities.append(row['entityType'])
                i += 1
        while i < sen_length:
            entities.append('O')
            i += 1
    return entities

In [18]:
for idx, row in sen_entities_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

Belton House : LOC
Belton : LOC
Grantham : LOC
Lincolnshire : LOC
England : LOC


In [19]:
entity_list = entityListFromSentence(senIdx=0, sen_length=len(dec_sentence),
                                     art_entities_df=art_entities_df)
print(' '.join(article_df['sentence'][1]))
print(entity_list)

Belton House is a Grade I listed country house in Belton near Grantham , Lincolnshire , England .
['LOC', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O', 'LOC', 'O']


In [20]:
def w2v_strategy(strategy, sentence, W, w2v_model):
    return {
        'concat': concat_vectors(sentence, W, w2v_model), 
        'mean': mean_vectors(sentence, W, w2v_model), 
        'frac_decay': fractional_decay(sentence, W, w2v_model),
        'exp_decay': exponential_decay(sentence, W, w2v_model),
    }[strategy]

In [21]:
def getVector_EntityFromArticle(article_df, art_entities_df, strategy, W, w2v_model):
    '''@return: filled DataFrame with columns {wordVector, entityType}'''
    article_df = article_df.reset_index(drop=True)
    article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))

    fun = lambda sentence: w2v_strategy(strategy, sentence, W, w2v_model)
    article_df['sen_vectors'] = article_df['sentence'].map(fun)
    art_vectors = list(article_df['sen_vectors'])

    fun2 = lambda senIdx: entityListFromSentence(senIdx, article_df.loc[senIdx, 'sen_length'],
                                                 art_entities_df)
    article_df['sen_entities'] = article_df.index.map(fun2)
    art_entities = list(article_df['sen_entities'])
    # Fastest way to flatten list of arrays
    art_vectors = list(itertools.chain(*art_vectors))
    art_entities = list(itertools.chain(*art_entities))

    wordVector_Entity_df = pd.DataFrame({'wordVector': art_vectors, 
                                         'entityType': art_entities})

    return wordVector_Entity_df

In [22]:
start = time.time()
wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                   strategy='exp_decay',
                                                   W=5, w2v_model=w2v_model)
end = time.time()
print('demora:', end-start)

demora: 10.182647228240967


In [23]:
entity_list = list(wordVector_Entity_df['entityType'])
tokens = len(entity_list)
print('Cantidad de palabras: {}'.format(tokens))
n_e = len(entity_list) - entity_list.count('O')
print('Cantidad de entidades nombradas: {}'.format(n_e))
print('Porcentaje de entidades nombradas: {:.2f}%'.format(n_e/tokens*100))

Cantidad de palabras: 4395
Cantidad de entidades nombradas: 399
Porcentaje de entidades nombradas: 9.08%


En general, cada artículo contiene una proporción reducida de palabras que son entidades.

Una alternativa para evitar cómputo y uso de memoria podría ser eliminar una proporción de vectores que no representan ninguna entidad (etiquetados con 'O') a la hora de construir la matriz de palabra - entidad.

In [24]:
def drop_non_entities(df, frac):
    '''
    Remove a fraction of non entities vectors (entityType == 'O')
    df: wordVector_Entity_df
    frac: float value between 0 and 1
    @return df with a fraction of the non entities rows removed
    '''
    sample = df[df.entityType == 'O'].sample(frac=frac, random_state=77)
    df = df.drop(index=sample.index)

    return df

In [25]:
wordVector_Entity_df.head(10)

,wordVector,entityType
0,"[-0.003621502994722617, 0.012132806459215266, ...",LOC
1,"[-0.08827082592044064, -0.13598437701614993, 0...",LOC
2,"[-0.16997208299104571, 0.0033914764131283102, ...",O
3,"[-0.3915967207881669, -0.12179563739202309, 0....",O
4,"[0.048302400391625866, -0.09540661652931712, 0...",O
5,"[-0.5453598294447486, -0.15736036498200667, 0....",O
6,"[-0.12074501704763917, 0.06705739051063955, 0....",O
7,"[-0.014825387882357826, -0.25825100262032796, ...",O
8,"[-0.10727509902624327, 0.04387436488263421, 0....",O
9,"[0.07985290764610813, -0.14920335335911, -0.03...",O


In [26]:
drop_non_entities(wordVector_Entity_df.head(10), 0.50)

,wordVector,entityType
0,"[-0.003621502994722617, 0.012132806459215266, ...",LOC
1,"[-0.08827082592044064, -0.13598437701614993, 0...",LOC
5,"[-0.5453598294447486, -0.15736036498200667, 0....",O
6,"[-0.12074501704763917, 0.06705739051063955, 0....",O
8,"[-0.10727509902624327, 0.04387436488263421, 0....",O
9,"[0.07985290764610813, -0.14920335335911, -0.03...",O


In [29]:
def genWordVectors_Entity(doc_filename, coarseNE_filename, strategy, W, w2v_model):
    '''
    Creates a N x D matrix of word vectors and saves it to disk. 
    Creates an 1 x N matrix of entities and saves it to disk. 
    N is the number of words in the document.
    D is the size of each word vector.
    The entity types are: PER - LOC - ORG - MISC - O
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    w2v_model: pre-trained word2vec model
    '''
    wordVectors = []
    entityVector = []
    count = 0 # Used in the 'progress bar'    
    doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_filename)
    coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_filename)
    art_IDs = coarseNE_df.art_ID.unique()      
    # We consider only the articles with at least one entity.
    # That's why we iterate over the coarseNE's articles.
    for art_ID in np.nditer(art_IDs):
        article_df = doc_df[doc_df.art_ID == art_ID]
        art_entities_df = coarseNE_df[coarseNE_df.art_ID == art_ID]     
        wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                           strategy, W, w2v_model)         
        wordVector_Entity_df = drop_non_entities(wordVector_Entity_df, 0.90)
        wordVectors += list(wordVector_Entity_df['wordVector'])         
        entityVector += list(wordVector_Entity_df['entityType'])

        if count % 10 == 0: # Kind of progress bar :P
            print(count, end=' ')
        count += 1

    starting = time.time()
    np.savez_compressed('./corpus_WiNER/entity_vectors/ev_' + doc_filename + '_' + strategy,
                        entityVector)
    np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+ doc_filename + '_' + strategy,
                        *wordVectors)
    finishing = time.time()
    print('tiempo de guardado:', finishing - starting)

In [28]:
# start = time.time()
# genWordVectors_Entity(doc_filename='doc_0', coarseNE_filename='coarseNE_0',
#                       strategy='exp_decay', W=5, w2v_model=w2v_model)
# end = time.time()
# print('Demora total: {}'.format(end - start))

In [19]:
# word_vectors = np.load('./corpus_WiNER/word_vectors/wv_doc_0_exp_decay_.npz')
# word_vectors = np.load('./corpus_WiNER/wv_doc_0_exp_decay_.npz')

In [ ]:
# for a_name, arr in word_vectors.iteritems():
#     print(a_name, len(arr))
#     break

In [ ]:
# entity_vector = np.load('./corpus_WiNER/entity_vectors/ev_doc_0_exp_decay_.npz')
# entity_vector = np.load('./corpus_WiNER/ev_doc_0_exp_decay_.npz')

In [ ]:
# for a_name, arr in entity_vector.iteritems():
#     print(a_name, len(arr))
#     entities = list(arr)

In [ ]:
# entities.count('O')

# División de los documentos

### Cargamos los primeros 1500 documentos que se van a utilizar para la parte supervisada

In [20]:
doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
doc_filenames.sort()
doc_filenames = doc_filenames[0:1500]
coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
coarseNE_filenames.sort()
coarseNE_filenames = coarseNE_filenames[0:1500]
docs = []
coarseNEs = []
for doc, ne in zip(doc_filenames, coarseNE_filenames):
    docs.append(pd.read_pickle('./corpus_WiNER/docs_df/'+ doc))
    coarseNEs.append(pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ ne))        

In [21]:
docs_df = pd.concat(docs, ignore_index=True)
coarseNE_df = pd.concat(coarseNEs, ignore_index=True)

In [22]:
print('Cantidad de oraciones:', docs_df.shape[0])

Cantidad de oraciones: 23811536


In [23]:
print('Cantidad de artículos que no contienen ninguna entidad:', 
      len(docs_df.art_ID.unique()) - len(coarseNE_df.art_ID.unique()))

Cantidad de artículos que no contienen ninguna entidad: 3449


Nos quedamos con aquellos artículos que contienen al menos una entidad nombrada.

In [24]:
art_IDs = coarseNE_df.art_ID.unique()

#### Extraemos una muestra aleatoria de 4000 artículos

In [25]:
np.random.seed(42) # No funciona ??
np.random.shuffle(art_IDs)
art_IDs_sample = art_IDs[0:4000]

#### Filtramos

In [26]:
articles_df = docs_df[docs_df.art_ID.isin(art_IDs_sample)]
entities_df = coarseNE_df[coarseNE_df.art_ID.isin(art_IDs_sample)]

In [27]:
articles_df.shape # Por mas que se setea el seed el sample va cambiando

(65993, 2)

In [28]:
def genWordVectors_Entity(doc_df, coarseNE_df, strategy, W, w2v_model):
    '''
    Creates a N x D matrix of word vectors and saves it to disk. 
    Creates an 1 x N matrix of entities and saves it to disk. 
    N is the number of words in the document.
    D is the size of each word vector.
    The entity types are: PER - LOC - ORG - MISC - O
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    w2v_model: pre-trained word2vec model
    '''
    wordVectors = []
    entityVector = []   
    art_IDs = coarseNE_df.art_ID.unique()      
    # We consider only the articles with at least one entity.
    # That's why we iterate over the coarseNE's articles.
    for art_ID in tqdm(np.nditer(art_IDs)):
        article_df = doc_df[doc_df.art_ID == art_ID]
        art_entities_df = coarseNE_df[coarseNE_df.art_ID == art_ID]     
        wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                           strategy, W, w2v_model)         
        wordVector_Entity_df = drop_non_entities(wordVector_Entity_df, 0.80)
        wordVectors += list(wordVector_Entity_df['wordVector'])         
        entityVector += list(wordVector_Entity_df['entityType'])

    starting = time.time()
    np.savez_compressed('./corpus_WiNER/entity_vectors/ev_' + 'sample' + '_' + strategy 
                        + '_W_' + str(W), entityVector)
    np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+ 'sample' + '_' + strategy
                        + '_W_' + str(W), wordVectors)
    finishing = time.time()
    print('tiempo de guardado:', finishing - starting)

In [29]:
start = time.time()
genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='exp_decay', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

0 10 

/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2220 2

In [30]:
start = time.time()
genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='frac_decay', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

0 10 

/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2220 2

In [31]:
start = time.time()
genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='mean', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

0 10 

/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2220 2

In [32]:
# start = time.time()
# genWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
#                       strategy='concat', W=5, w2v_model=w2v_model)
# end = time.time()
# print('Demora total: {}'.format(end - start))

### Cargamos los segundos 1500 documentos que se van a utilizar para la parte no supervisada

In [31]:
doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
doc_filenames.sort()
doc_filenames = doc_filenames[1500:3000]
coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
coarseNE_filenames.sort()
coarseNE_filenames = coarseNE_filenames[1500:3000]
docs = []
coarseNEs = []
for doc, ne in zip(doc_filenames, coarseNE_filenames):
    docs.append(pd.read_pickle('./corpus_WiNER/docs_df/'+ doc))
    coarseNEs.append(pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ ne))        

In [32]:
docs_df = pd.concat(docs, ignore_index=True)
coarseNE_df = pd.concat(coarseNEs, ignore_index=True)

In [33]:
print('Cantidad de oraciones:', docs_df.shape[0])

Cantidad de oraciones: 25935452


In [34]:
print('Cantidad de artículos que no contienen ninguna entidad:', 
      len(docs_df.art_ID.unique()) - len(coarseNE_df.art_ID.unique()))

Cantidad de artículos que no contienen ninguna entidad: 2732


Nos quedamos con aquellos artículos que contienen al menos una entidad nombrada.

In [35]:
art_IDs = coarseNE_df.art_ID.unique()

#### Extraemos una muestra aleatoria de 12000 artículos

In [36]:
np.random.seed(42) # No funciona ??
np.random.shuffle(art_IDs)
art_IDs_sample = art_IDs[0:12000]

#### Filtramos

In [37]:
articles_df = docs_df[docs_df.art_ID.isin(art_IDs_sample)]
entities_df = coarseNE_df[coarseNE_df.art_ID.isin(art_IDs_sample)]

In [38]:
articles_df.shape # Por mas que se setea el seed el sample va cambiando

(201934, 2)

In [41]:
def genNonSupervisedWordVectors_Entity(doc_df, coarseNE_df, strategy, W, w2v_model):
    '''
    Creates a N x D matrix of word vectors and saves it to disk. 
    Creates an 1 x N matrix of entities and saves it to disk. 
    N is the number of words in the document.
    D is the size of each word vector.
    The entity types are: PER - LOC - ORG - MISC - O
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    w2v_model: pre-trained word2vec model
    '''
    wordVectors = []
    entityVector = []   
    art_IDs = coarseNE_df.art_ID.unique()      
    # We consider only the articles with at least one entity.
    # That's why we iterate over the coarseNE's articles.
    for art_ID in tqdm(np.nditer(art_IDs)):
        article_df = doc_df[doc_df.art_ID == art_ID]
        art_entities_df = coarseNE_df[coarseNE_df.art_ID == art_ID]     
        wordVector_Entity_df = getVector_EntityFromArticle(article_df, art_entities_df, 
                                                           strategy, W, w2v_model)         
        wordVector_Entity_df = drop_non_entities(wordVector_Entity_df, 0.80)
        wordVectors += list(wordVector_Entity_df['wordVector'])         
        entityVector += list(wordVector_Entity_df['entityType'])

    starting = time.time()
    np.savez_compressed('./corpus_WiNER/entity_vectors/ev_' + 'no_supervised' + '_' + strategy 
                        + '_W_' + str(W), entityVector)
    np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+ 'no_supervised' + '_' + strategy
                        + '_W_' + str(W), wordVectors)
    finishing = time.time()
    print('tiempo de guardado:', finishing - starting)

In [42]:
start = time.time()
genNonSupervisedWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='exp_decay', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

116it [01:20,  1.44it/s]                     /users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
12000it [3:18:01,  1.01it/s]


tiempo de guardado: 672.1255185604095
Demora total: 12557.627238988876


In [43]:
start = time.time()
genNonSupervisedWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='frac_decay', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

116it [01:31,  1.27it/s]                     /users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
12000it [3:29:47,  1.05s/it]


tiempo de guardado: 1298.1283988952637
Demora total: 13889.372932195663


In [44]:
start = time.time()
genNonSupervisedWordVectors_Entity(doc_df=articles_df, coarseNE_df=entities_df,
                      strategy='mean', W=5, w2v_model=w2v_model)
end = time.time()
print('Demora total: {}'.format(end - start))

115it [01:17,  1.48it/s]                     /users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/users/ekokic/thesis_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
12000it [3:29:09,  1.05s/it]


tiempo de guardado: 1099.0262243747711
Demora total: 13652.744041204453
